In [23]:
num_sample = 300
num_burn = 20
sample_size = 800
n_cpu = 10

min_degree = 3
max_degree = 10

In [24]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
adj = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj.sum(axis=1).min(), adj.sum(axis=1).max())

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
# gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
# beta = np.array([-1*min_degree,2,-0.2,2,0.1,-2,0.1,2,0.1,0.1])  # shape (10,)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1*min_degree,2*min_degree,-2,2,0.1,-2,0.1,2,0.1,0]) # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2, dir3, dir4 = [], [], [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(np.mean(Y[A==1]) - Y[A==0]))
    dir2.append(np.mean(A))
    dir3.append(np.mean(A[L[:,0]==1]) - np.mean(A[L[:,0]==0]))
    dir4.append(np.mean(Y[L[:,0]==1]) - np.mean(Y[L[:,0]==0]))

np.mean(dir), np.mean(dir2), np.mean(Y_chain), np.mean(dir3), np.mean(dir4)

3 10


100%|██████████| 320/320 [00:15<00:00, 20.55it/s]


(0.27446584111291494,
 0.5821875000000001,
 0.1635,
 0.32397469144445507,
 0.16539988582795498)

In [25]:
agcEffect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=100,
    burnin_R=100,
    seed=0,
    sample_Y_func=sample_Y1
)

100%|██████████| 200/200 [00:06<00:00, 32.71it/s]


{'average': 0.14117499999999997,
 'direct_effect': 0.19323749999999995,
 'spillover_effect': -0.22485,
 'psi_1_gamma': 0.19976249999999995,
 'psi_0_gamma': 0.0065249999999999996,
 'psi_zero': 0.231375}

In [26]:
from utils import run_pll
from run_pll import run_autognet

np.random.seed(0)
L_chain_noise = np.random.binomial(n=1, p=0.2, size=L_chain.shape)
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain_noise,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)


Multiprocessing <function run_autognet at 0x12eef4e50> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:01<00:00, 30.28it/s]


Multiprocessing finished.


In [27]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.18931 ± 0.01024
direct_effect: 0.26845 ± 0.01575
spillover_effect: -0.06775 ± 0.09121
psi_1_gamma: 0.27044 ± 0.01603
psi_0_gamma: 0.00199 ± 0.00193
psi_zero: 0.06974 ± 0.09246


{'average': 0.41311249999999994,
 'direct_effect': 0.27885000000000015,
 'spillover_effect': -0.1828625,
 'psi_1_gamma': 0.4988250000000001,
 'psi_0_gamma': 0.21997499999999995,
 'psi_zero': 0.40283749999999996}

In [28]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i,} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)


Multiprocessing <function run_autognet at 0x12eef4e50> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 361.85it/s]


Multiprocessing finished.


In [29]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.14187 ± 0.01077
direct_effect: 0.19297 ± 0.01701
spillover_effect: -0.21496 ± 0.17932
psi_1_gamma: 0.19969 ± 0.01550
psi_0_gamma: 0.00672 ± 0.00624
psi_zero: 0.22168 ± 0.18392


In [30]:
# from run_pll import *
# from utils import run_pll

# args = [{'Y_chain': Y_chain,
#          'A_chain': A_chain,
#          'L_chain': L_chain,
#          'adj': adj,
#          'i': i,
#          'mispec': 'outcome'} for i in range(len(Y_chain))]

# res_list_dr = run_pll(run_dr, args, processes=n_cpu)
# res_list_array_dr = np.array(res_list_dr)

# from run_pll import column_names

# ret_mean_dr = res_list_array_dr.mean(axis=0)
# ret_std_dr = res_list_array_dr.std(axis=0)
# for i in range(len(column_names)):
#     print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

In [31]:
# from run_pll import *
# from utils import run_pll

# args = [{'Y_chain': Y_chain,
#          'A_chain': A_chain,
#          'L_chain': L_chain,
#          'adj': adj,
#          'i': i,
#          'mispec': 'treatment'} for i in range(len(Y_chain))]

# res_list_dr = run_pll(run_dr, args, processes=n_cpu)
# res_list_array_dr = np.array(res_list_dr)

# from run_pll import column_names

# ret_mean_dr = res_list_array_dr.mean(axis=0)
# ret_std_dr = res_list_array_dr.std(axis=0)
# for i in range(len(column_names)):
#     print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")